In [1]:
import bs4 as bs
import requests
import pandas as pd
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [18]:
import warnings
import logging


warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)


def scrap(url, chrome_driver_path, N_SCROLL, output_path="", to_csv=False):
    import time
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--lang=id")
    service = Service('chromedriver.exe')

    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=service)
    logging.info("opening url..")
    
    driver.get(url)
    
    wait = WebDriverWait(driver, 20)

    ## Use this code below if you use url of the start page on gmaps, not on reviews. This button will click review
    logging.info("Open review tab if not yet opened..")
    try:
        menu_review = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]')))
        menu_review.click()
    except :
        pass # Pass if already opened

    logging.info("click terbaru..")
    try:
        menu_old = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde > div.m6QErb.Pf6ghf.XiKgde.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc > button')))
        menu_old.click()
    except:
        # menu = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[9]/button[2]')))
        menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
        '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde > div.m6QErb.Hk4XGb.XiKgde.tLjsW > button:nth-child(2)')))
        menu.click()   


    terbaru = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="action-menu"]/div[2]')))
    terbaru.click()

    time.sleep(1)

    # scroll down
    logging.info("scrolling down..")
    
    scrollable_div = driver.find_element(By.CSS_SELECTOR, 
    '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf'
    )

    for _ in range(N_SCROLL):
        driver.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollHeight',
            scrollable_div
        )
        time.sleep(1)

    # Scraping
    logging.info("get html..")
    resp = bs.BeautifulSoup(driver.page_source, 'lxml')
    reviews = resp.select(
        "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde > div:nth-last-child(2)")

    soup_reviews = bs.BeautifulSoup(str(reviews), 'lxml')
    review = soup_reviews.find_all('div', class_='jftiEf fontBodyMedium')

    # Nama
    logging.info("get nama..")
    names = []
    for i in range(len(review)):
        names.append(review[i]['aria-label'])

    # Bintang
    logging.info("get bintang..")
    soup_bapak_bintang = bs.BeautifulSoup(str(review), 'lxml')
    bapak_bintang = soup_bapak_bintang.find_all('div', class_='DU9Pgb')

    bintangs = []
    for i in range(len(bapak_bintang)):
        soup_bintang = bs.BeautifulSoup(str(bapak_bintang[i]), 'lxml')
        bintang = soup_bintang.find('span', class_='fzvQIb')
        bintang_text = bintang.text.strip().split('/')[0]
        bintangs.append(bintang_text)

    # Komentar
    logging.info("get komentar..")
    komens = []
    for i in range(len(review)):
        soup = bs.BeautifulSoup(str(review[i]), 'lxml')
        span = soup.find('span', class_="wiI7pd")
        if span:
            komens.append(span.text)
        else:
            komens.append("")  

    soup_bapak_time = bs.BeautifulSoup(str(review), 'lxml')
    bapak_time = soup_bapak_time.find_all('div', class_='DU9Pgb')

    times = []
    for i in range(len(bapak_time)):
        soup_time = bs.BeautifulSoup(str(bapak_time[i]), 'lxml')
        time = soup_time.find('span', class_='xRkPPb')
        time_text = time.text.strip().split(' ago')[0]
        times.append(time_text)

    data = pd.DataFrame({
        "nama": names,
        "bintang": bintangs,
        "komentar": komens,
        "Waktu" : times
    })  

    logging.info("to csv..")
    if to_csv:
        data.to_csv(output_path, index=False)
    else:
        return data

In [19]:
urls = [
    # "https://www.google.com/maps/place/Bobopod+Alun-Alun,+Malang/@-7.9835318,112.6301217,17z/data=!4m26!1m13!3m12!1s0x2dd629833fc56943:0x6a25edef151ebe41!2sBobopod+Alun-Alun,+Malang!5m3!1s2024-07-04!4m1!1i2!8m2!3d-7.9835318!4d112.6301217!9m1!1b1!16s%2Fg%2F11sx2hc0m9!3m11!1s0x2dd629833fc56943:0x6a25edef151ebe41!5m3!1s2024-07-04!4m1!1i2!8m2!3d-7.9835318!4d112.6301217!9m1!1b1!16s%2Fg%2F11sx2hc0m9?entry=ttu",
    "https://maps.app.goo.gl/JDdWiXkGWVtnvpJt6",
    "https://www.google.com/maps/place/Bobopod+Alun-Alun,+Bandung/@-6.9233953,107.6021185,17z/data=!4m12!3m11!1s0x2e68e75ed3eddc13:0xd79636630fb365e9!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.9234006!4d107.6046934!9m1!1b1!16s%2Fg%2F11j3rx32hd?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Paskal,+Bandung/@-6.910383,107.5980516,17z/data=!4m12!3m11!1s0x2e68e7a91c1617e5:0x97ce609bb624b1be!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.910383!4d107.5980516!9m1!1b1!16s%2Fg%2F11f62jqb_t?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Cipaganti,+Bandung/@-6.9028047,107.6002061,17z/data=!4m12!3m11!1s0x2e68e64157efffff:0x88fd0151f5a0a4a5!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.90281!4d107.602781!9m1!1b1!16s%2Fg%2F11h1fwkqpf?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Mega+Mall,+Bekasi/@-6.2484929,106.9875422,15z/data=!4m12!3m11!1s0x2e698d4035e8c6f7:0x7f4228434c108d52!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2490691!4d106.9926926!9m1!1b1!16s%2Fg%2F11trsfzc3k?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Kota+Tua,+Jakarta/@-6.1349894,106.8104188,17z/data=!4m12!3m11!1s0x2e6a1d858aef2f2d:0xc60c0722d0b726a2!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.134056!4d106.8131332!9m1!1b1!16s%2Fg%2F11p5zjpvsx?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Thamrin,+Jakarta/@-6.2017388,106.8234067,17z/data=!4m26!1m13!3m12!1s0x2e69f51581a992d9:0x4e0281dfccb4c5e1!2sBobopod+Thamrin,+Jakarta!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2017388!4d106.8234067!9m1!1b1!16s%2Fg%2F11l350_5lw!3m11!1s0x2e69f51581a992d9:0x4e0281dfccb4c5e1!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2017388!4d106.8234067!9m1!1b1!16s%2Fg%2F11l350_5lw?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Juanda,+Jakarta/@-6.1672761,106.8282771,17z/data=!4m12!3m11!1s0x2e69f51aac80b18f:0x8f74127e0bdb2d8d!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.1672814!4d106.830852!9m1!1b1!16s%2Fg%2F11km_cnzpq?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Kota+Lama,+Semarang/@-6.9688119,110.4238114,17z/data=!4m26!1m13!3m12!1s0x2e70f35621045bc9:0x6275da72438bafab!2sBobopod+Kota+Lama,+Semarang!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.9688172!4d110.4263863!9m1!1b1!16s%2Fg%2F11fs8s889p!3m11!1s0x2e70f35621045bc9:0x6275da72438bafab!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.9688172!4d110.4263863!9m1!1b1!16s%2Fg%2F11fs8s889p?entry=ttu",    
    "https://www.google.com/maps/place/Bobobox+Pods+Solo+Slamet+Riyadi/@-7.5704963,110.821215,17z/data=!4m10!3m9!1s0x2e7a16624b1bffff:0x5b83879ff8d9849b!5m3!1s2024-07-04!4m1!1i2!8m2!3d-7.5705016!4d110.8237899!16s%2Fg%2F11kk_41nwf?entry=tts&g_ep=EgoyMDI0MDcwMS4wKgBIAVAD",
    "https://www.google.com/maps/place/Bobopod+Airport+CBC,+Tangerang/@-6.1108815,106.688556,17z/data=!4m11!3m10!1s0x2e6a03c5861aebc1:0x9880e899710c7d4a!5m2!4m1!1i2!8m2!3d-6.1108868!4d106.6911309!9m1!1b1!16s%2Fg%2F11p5zjpl44?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Malioboro,+Yogyakarta/@-7.794984,110.3627654,17z/data=!4m10!3m9!1s0x2e7a5827ebaeaaab:0xa7badf5804f1716b!5m3!1s2024-07-04!4m1!1i2!8m2!3d-7.7949893!4d110.3653403!16s%2Fg%2F11jk1j1mx9?entry=tts&g_ep=EgoyMDI0MDcwMS4wKgBIAVAD",    
    "https://www.google.com/maps/place/Bobocabin+Cikole,+Bandung/@-6.7840681,107.6479342,17z/data=!4m12!3m11!1s0x2e68e1f0b0e7dfa5:0xaf16c818ec27010c!5m3!1s2024-07-15!4m1!1i2!8m2!3d-6.7840681!4d107.6479342!9m1!1b1!16s%2Fg%2F11q1m48kt3?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Pangalengan,+Bandung/@-7.1723274,107.5274359,17z/data=!4m12!3m11!1s0x2e6893af796033b7:0x8256c239defbbd31!5m3!1s2024-07-15!4m1!1i2!8m2!3d-7.1723274!4d107.5274359!9m1!1b1!16s%2Fg%2F11stzs3r4j?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Baturraden,+Purwokerto/@-7.3075366,109.2351977,17z/data=!4m12!3m11!1s0x2e6ff5e337322fbd:0x2eac513b8ddaf3ef!5m3!1s2024-07-15!4m1!1i2!8m2!3d-7.3075366!4d109.2351977!9m1!1b1!16s%2Fg%2F11tc30s51f?entry=ttu",    
    "https://www.google.com/maps/place/Bobocabin+Gunung+Mas,+Puncak/@-6.7061385,106.9690882,17z/data=!4m26!1m13!3m12!1s0x2e69b7fbf88bb96f:0x2873382678b0c488!2sBobocabin+Gunung+Mas,+Puncak!5m3!1s2024-07-15!4m1!1i2!8m2!3d-6.7061385!4d106.9690882!9m1!1b1!16s%2Fg%2F11s39bs8k8!3m11!1s0x2e69b7fbf88bb96f:0x2873382678b0c488!5m3!1s2024-07-15!4m1!1i2!8m2!3d-6.7061385!4d106.9690882!9m1!1b1!16s%2Fg%2F11s39bs8k8?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Dieng,+Wonosobo/@-7.1334089,109.9033429,17z/data=!4m26!1m13!3m12!1s0x2e70134de10883f1:0x739455cb25b3fb37!2sBobocabin+Dieng,+Wonosobo!5m3!1s2024-07-15!4m1!1i2!8m2!3d-7.1334089!4d109.9033429!9m1!1b1!16s%2Fg%2F11vbrns5w2!3m11!1s0x2e70134de10883f1:0x739455cb25b3fb37!5m3!1s2024-07-15!4m1!1i2!8m2!3d-7.1334089!4d109.9033429!9m1!1b1!16s%2Fg%2F11vbrns5w2?authuser=0&hl=en&entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Gunung+Rinjani,+Lombok/@-8.3567277,116.5113991,17z/data=!4m12!3m11!1s0x2dcc2d838d6ac069:0x27d80bf59a208e7!5m3!1s2024-07-15!4m1!1i2!8m2!3d-8.3561625!4d116.5134905!9m1!1b1!16s%2Fg%2F11l1yqk8lh?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Kintamani,+Bali/@-8.2353253,115.4003266,17z/data=!4m26!1m13!3m12!1s0x2dd1f7dd87b03d29:0x31b4d2c53cffa12b!2sBobocabin+Kintamani,+Bali!5m3!1s2024-07-15!4m1!1i2!8m2!3d-8.2353253!4d115.4003266!9m1!1b1!16s%2Fg%2F11rdbf5v39!3m11!1s0x2dd1f7dd87b03d29:0x31b4d2c53cffa12b!5m3!1s2024-07-15!4m1!1i2!8m2!3d-8.2353253!4d115.4003266!9m1!1b1!16s%2Fg%2F11rdbf5v39?entry=ttu",
    # "https://www.google.com/maps/place/Bobocabin+Bunaken+Hills,+Manado/@1.5464254,119.5559234,6z/data=!4m11!3m10!1s0x32879f01d075addb:0xdf5292b606f05b5c!5m2!4m1!1i2!8m2!3d1.5464254!4d124.8293609!9m1!1b1!16s%2Fg%2F11vhj2jlz1?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Coban+Rondo,+Malang/@-7.8703726,112.4867719,17z/data=!4m11!3m10!1s0x2e7887701316f659:0x14e6735e07f2c919!5m2!4m1!1i2!8m2!3d-7.8703726!4d112.4867719!9m1!1b1!16s%2Fg%2F11t6n8dv4x?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Pacet,+Mojokerto/@-7.7764156,112.4361679,12z/data=!4m12!3m11!1s0x2e78797f02391a95:0xea60e78437cb175b!5m3!1s2024-07-06!4m1!1i2!8m2!3d-7.6828146!4d112.5503686!9m1!1b1!16s%2Fg%2F11thdsjclm?entry=ttu",
    # "https://www.google.com/maps/place/Bobocabin+Madasari,+Pangandaran/@-7.7876668,108.4985918,17z/data=!4m26!1m13!3m12!1s0x2e65bf6f7a9dde51:0x77c5657a3992c608!2sBobocabin+Madasari,+Pangandaran!5m3!1s2024-07-06!4m1!1i2!8m2!3d-7.7876668!4d108.4985918!9m1!1b1!16s%2Fg%2F11ssznvh8z!3m11!1s0x2e65bf6f7a9dde51:0x77c5657a3992c608!5m3!1s2024-07-06!4m1!1i2!8m2!3d-7.7876668!4d108.4985918!9m1!1b1!16s%2Fg%2F11ssznvh8z?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Umarato,+Sumba/@-9.7534168,119.413178,17z/data=!4m12!3m11!1s0x2c4afd2a9be0614f:0x8429f2c37c7984c5!5m3!1s2024-07-06!4m1!1i2!8m2!3d-9.7534168!4d119.413178!9m1!1b1!16s%2Fg%2F11tst_bgc7?entry=ttu",
    "https://www.google.com/maps/place/BOBOCABIN+SIGNATURE+TOBA/@2.6070585,98.9461666,17z/data=!4m12!3m11!1s0x3031efb97c62223f:0x2be86268c92a66e5!5m3!1s2024-07-06!4m1!1i2!8m2!3d2.6070611!4d98.9461639!9m1!1b1!16s%2Fg%2F11t0wb8866?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Ubud,+Bali/@-8.4193463,115.2894999,17z/data=!4m26!1m13!3m12!1s0x2dd219f47031027f:0xd6407a12af23392a!2sBobocabin+Ubud,+Bali!5m3!1s2024-07-06!4m1!1i2!8m2!3d-8.4193463!4d115.2894999!9m1!1b1!16s%2Fg%2F11jzs0rwxp!3m11!1s0x2dd219f47031027f:0xd6407a12af23392a!5m3!1s2024-07-06!4m1!1i2!8m2!3d-8.4193463!4d115.2894999!9m1!1b1!16s%2Fg%2F11jzs0rwxp?entry=ttu",
    # "https://www.google.com/maps/place/Bobocabin+Sukawana+Bandung/@-6.7797699,107.5885735,20z/data=!4m22!1m10!3m9!1s0x2e68e1edd84db0c7:0x394be4061e1fa27e!2sBobocabin+Sukawana+Bandung!5m2!4m1!1i2!8m2!3d-6.7797263!4d107.5887655!16s%2Fg%2F11w3kxr1x7!3m10!1s0x2e68e1edd84db0c7:0x394be4061e1fa27e!5m2!4m1!1i2!8m2!3d-6.7797263!4d107.5887655!9m1!1b1!16s%2Fg%2F11w3kxr1x7?entry=ttu",
    "https://www.google.com/maps/place/Bobocabin+Ranca+Upas,+Bandung/@-7.1396694,107.3908944,17z/data=!4m11!3m10!1s0x2e688d760cbb3e29:0x8a6b20171ca52dbc!5m2!4m1!1i2!8m2!3d-7.1396694!4d107.3908944!9m1!1b1!16s%2Fg%2F11r_g3g5g6?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Dago,+Bandung/@-6.9026546,107.6129526,15z/data=!4m12!3m11!1s0x2e68e7081539b8f5:0x481025495173e682!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.9023738!4d107.6129795!9m1!1b1!16s%2Fg%2F11hz28sy_d?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Tanah+Abang,+Jakarta/@-6.175195,106.8098294,17z/data=!4m12!3m11!1s0x2e69f7a0918ee1b3:0xdbabcdeda583f102!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.1752003!4d106.8124043!9m1!1b1!16s%2Fg%2F11kdkwp98t?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+ITC+Kuningan,+Jakarta/@-6.2238104,106.8238125,17z/data=!4m26!1m13!3m12!1s0x2e69f334238f8ba9:0x3b47d61e0aaa6b68!2sBobopod+ITC+Kuningan,+Jakarta!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2238157!4d106.8263874!9m1!1b1!16s%2Fg%2F11s2y24pxx!3m11!1s0x2e69f334238f8ba9:0x3b47d61e0aaa6b68!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2238157!4d106.8263874!9m1!1b1!16s%2Fg%2F11s2y24pxx?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Pancoran,+Jakarta/@-6.2465971,106.8398229,17z/data=!4m12!3m11!1s0x2e69f3a5aebd7ded:0xd6dce549aad7c0b9!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2466024!4d106.8423978!9m1!1b1!16s%2Fg%2F11j0bdyh10?entry=ttu",
    "https://www.google.com/maps/place/Bobopod+Kebayoran+Baru,+Jakarta/@-6.2493232,106.7792763,15z/data=!4m26!1m13!3m12!1s0x2e69f1eddef77381:0xca6fd86b6eb71e6!2sBobopod+Kebayoran+Baru,+Jakarta!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2493449!4d106.7977304!9m1!1b1!16s%2Fg%2F11h5mfwrqq!3m11!1s0x2e69f1eddef77381:0xca6fd86b6eb71e6!5m3!1s2024-07-04!4m1!1i2!8m2!3d-6.2493449!4d106.7977304!9m1!1b1!16s%2Fg%2F11h5mfwrqq?entry=ttu"
]

tempat = [
    # "Bobopod_Alun_Alun_Malang",
    "Bobocabin_The_Tavia_Puncak",
    "Bobopod_Alun_Alun_Bandung",
    "Bobopod_Paskal_Bandung",
    "Bobopod_Cipaganti_Bandung",
    "Bobopod_Mega_Mall_Bekasi",
    "Bobopod_Kota_Tua_Jakarta",
    "Bobopod_Thamrin_Jakarta",
    "Bobopod_Juanda_Jakarta",
    "Bobopod_Kota_Lama_Semarang",
    "Bobopod_Slamet_Riyadi_Solo",
    "Bobopod_Airport_CBC_Tangerang",
    "Bobopod_Malioboro_Yogyakarta",
    "Bobocabin_Cikole_Bandung",
    "Bobocabin_Pangalengan_Bandung",
    "Bobocabin_Baturraden_Purwokerto",
    "Bobocabin_Gunung_Mas_Puncak",
    "Bobocabin_Dieng_Wonosobo",
    "Bobocabin_Gunung_Rinjani_Lombok",
    "Bobocabin_Kintamani_Bali",
    # "Bobocabin_Bunaken_Hills_Manado",
    "Bobocabin_Coban_Rondo_Malang",
    "Bobocabin_Pacet_Mojokerto",
    # "Bobocabin_Madasari_Pangandaran",
    "Bobocabin_Umarato_Sumba",
    "Bobocabin_signature_Toba",
    "Bobocabin_Ubud_Bali",
    # "Bobocabin_Sukawana_Bandung",
    "Bobocabin_Ranca_Upas_Bandung",
    "Bobopod_Dago_Bandung",
    "Bobopod_Tanah_Abang_Jakarta",
    "Bobopod_ITC_Kuningan",
    "Bobopod_Pancoran_Jakarta",
    "Bobopod_Kebayoran_Baru_Jakarta"
]

for tempat, url in zip(tempat, urls):
    print(tempat)
    scrap(url, "../../chromedriver.exe", 100, output_path=f"data_temp/{tempat}.csv", to_csv=True)

Bobocabin_The_Tavia_Puncak


INFO:root:opening url..
INFO:root:Open review tab if not yet opened..
INFO:root:click terbaru..
INFO:root:scrolling down..
INFO:root:get html..
INFO:root:get nama..
INFO:root:get bintang..
INFO:root:get komentar..
INFO:root:to csv..


Bobopod_Alun_Alun_Bandung


INFO:root:opening url..
INFO:root:Open review tab if not yet opened..
INFO:root:click terbaru..
INFO:root:scrolling down..
INFO:root:get html..
INFO:root:get nama..
INFO:root:get bintang..
INFO:root:get komentar..
INFO:root:to csv..


Bobopod_Paskal_Bandung


INFO:root:opening url..
INFO:root:Open review tab if not yet opened..
INFO:root:click terbaru..
INFO:root:scrolling down..
